In [35]:
import pandas as pd
import numpy as np
import matplotlib as plt
from sklearn.model_selection import train_test_split

testData = pd.read_csv("test.csv")
testY = pd.read_csv("output.csv")
testData['stroke'] = testY['stroke']
trainData = pd.read_csv("train.csv")

In [36]:
#Filling Missing BMI Data === train

temp = trainData['bmi'].mean() + trainData['bmi'].median()
temp /= 2
temp
trainData['bmi'] = trainData['bmi'].fillna(temp)

In [37]:
#Filling Missing BMI Data ===test

temp = testData['bmi'].mean() + testData['bmi'].median()
temp /= 2
temp
testData['bmi'] = testData['bmi'].fillna(temp)

In [38]:
print( "Empty Smoking Status(Train) in " , 
     len( list( filter( lambda x: x == True , pd.isna(trainData['smoking_status']) ) ) )
     )


print( "Empty Smoking Status(Test) in " , 
     len( list( filter( lambda x: x == True , pd.isna(testData['smoking_status']) ) ) )
     )

Empty Smoking Status(Train) in  13292
Empty Smoking Status(Test) in  5751


In [39]:
# Segregating into Two Models

trainDataNS = trainData[ trainData['smoking_status'].isna() ]
trainDataNS = trainDataNS.drop( columns=['smoking_status'] )
trainDataS = trainData[ trainData['smoking_status'].notna() ]

testDataNS = testData[ testData['smoking_status'].isna() ]
testDataNS = testDataNS.drop( columns=['smoking_status'] )
testDataS = testData[ testData['smoking_status'].notna() ]

In [40]:
print( "Imbalance in train is ", trainDataS['stroke'].value_counts() )
print( "Imbalance in test is ", testDataS['stroke'].value_counts() )

Imbalance in train is  0    29470
1      638
Name: stroke, dtype: int64
Imbalance in test is  0    9443
1    3407
Name: stroke, dtype: int64


In [41]:
# X and Y division

yTrainDataNS = trainDataNS['stroke']
xTrainDataNS = trainDataNS.drop( columns=['stroke'] )

yTrainDataS = trainDataS['stroke']
xTrainDataS = trainDataS.drop( columns=['stroke'] )

In [42]:
yTestDataNS = testDataNS['stroke']
xTestDataNS = testDataNS.drop( columns=['stroke'] )

yTestDataS = testDataS['stroke']
xTestDataS = testDataS.drop( columns=['stroke'] )

In [43]:
#Nominal Categories to one-hot encoding === Train

#Smoke Data
xTrainDataS = pd.get_dummies( xTrainDataS, 
                             columns=['gender', 'ever_married', 'work_type' , 'Residence_type', 'smoking_status']
                            , prefix=['sex', 'married', 'work_type' , 'res_type', 'smoke'] )


#No Smoke Data
xTrainDataNS = pd.get_dummies( xTrainDataNS,
                              columns=['gender', 'ever_married', 'work_type' , 'Residence_type']
                            , prefix=['sex', 'married', 'work_type' , 'res_type'] )



In [44]:
#Nominal Categories to one-hot encoding ===Test

#Smoke Data
xTestDataS = pd.get_dummies( xTestDataS,
                            columns=['gender', 'ever_married', 'work_type' , 'Residence_type', 'smoking_status']
                            , prefix=['sex', 'married', 'work_type' , 'res_type', 'smoke'] )


#No Smoke Data
xTestDataNS = pd.get_dummies( xTestDataNS,
                             columns=['gender', 'ever_married', 'work_type' , 'Residence_type']
                            , prefix=['sex', 'married', 'work_type' , 'res_type'] )

In [60]:
from imblearn.over_sampling import SMOTE
smote = SMOTE(random_state=0)

xTrainDataNS, yTrainDataNS = smote.fit_sample(xTrainDataNS, yTrainDataNS)
xTrainDataS, yTrainDataS = smote.fit_sample(xTrainDataS, yTrainDataS)


In [61]:
from sklearn.model_selection import train_test_split

#Train Splitter

x_train, x_test, y_train, y_test = train_test_split(xTrainDataNS, yTrainDataNS, test_size=0.35, random_state=0)

x_trainS, x_testS, y_trainS, y_testS = train_test_split(xTrainDataS, yTrainDataS, test_size=0.35, random_state=0)

In [62]:
#Decison Tree

from sklearn import tree
clf = tree.DecisionTreeClassifier()

#No Smoke
clf = clf.fit(x_train, y_train)
#Smoke
# clfS = clf.fit(x_trainS, y_trainS)

In [63]:
predictions = clf.predict(x_test)
clf.score(x_test, y_test)

0.9855481908073455

In [64]:
from sklearn import metrics

metrics.confusion_matrix(y_test, predictions)

array([[4498,   80],
       [  53, 4572]])

In [65]:
#Testing
predictions = clf.predict(xTestDataNS)
print(clf.score(xTestDataNS, yTestDataNS))

metrics.confusion_matrix(yTestDataNS, predictions)

0.7219613980177361


array([[4121,   86],
       [1513,   31]])

In [59]:
###XGBOOST

import xgboost as xgb

data_dmatrix = xgb.DMatrix(data=xTrainDataNS,label=yTrainDataNS)


cv_results = xgb.cv(dtrain=data_dmatrix, params=params, nfold=3,
                    num_boost_round=50,early_stopping_rounds=10,metrics="rmse", as_pandas=True, seed=123)

cv_results.head()


ModuleNotFoundError: No module named 'xgboost'